Description: This project is a machine learning-based support ticket classification system designed to categorize customer queries into predefined categories like login issues, app functionality, billing, account management, and performance issues. It leverages Sentence Transformers for embedding-based retrieval and the Llama 3.1 model for generative classification. The system features a user-friendly Gradio interface for inputting support tickets and ensures that irrelevant or out-of-scope queries are handled gracefully by providing a custom fallback message, which prevents the hallucinations. This solution is ideal for automating and enhancing customer support workflows.

Author: Riya Chougule

Date: 08/18/2024

In [1]:
import gradio as gr
import torch
import nltk
from sentence_transformers import SentenceTransformer, util
import ollama

# Load Sentence Transformer for embedding the knowledge base
embedder = SentenceTransformer('all-MiniLM-L6-v2')


/Users/riya/Downloads/Edlight-classification-task/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/riya/Downloads/Edlight-classification-task/myenv/lib/python3.9/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/Users/riya/Downloads/Edlight-classification-task/myenv/lib/python3.9/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/Users/riya/Downloads/Edlight-classification-task/myenv/lib/p

RAG component

In [32]:
# Knowledge base/content store
knowledge_base = [
    "Login Issues: Login issues often occur due to incorrect passwords or account lockouts.",
    "App Functionality: App crashes can be caused by outdated software or device incompatibility.",
    "Billing: Billing discrepancies may result from processing errors or duplicate transactions.",
    "Account Management: Account management includes tasks such as changing, updating, modifying profile information, linking social media accounts, and managing privacy settings.",
    "Performance Issues: Performance issues can be related to device specifications, network connectivity, or app optimization.",
    "No Relevant Information: It looks like I don’t have the relevant information you’re looking for. For further assistance, please contact our support team directly at techsupport@company.com"
]

# Embed the knowledge base
kb_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True)

def classify_ticket(Ticket):
    """
    Classifies a support ticket into a predefined category using a combination of retrieval and language model generation.

    Parameters:
        Ticket (str): The support ticket text to be classified.

    Returns:
        str: The classification result, either as one of the predefined categories or a custom message for irrelevant information.
    """

    query_embedding = embedder.encode(Ticket, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, kb_embeddings)[0]
    top_k_idx = torch.topk(scores, k=1).indices[0].item()
    top_k_score = scores[top_k_idx].item()
    
    # Define a similarity threshold
    similarity_threshold = 0.1
    
    if top_k_score < similarity_threshold:
        # Return the message for the new category 6
        return knowledge_base[-1]
    
    retrieved_text = knowledge_base[top_k_idx]
    prompt = f"Support Ticket: {Ticket}\nRelevant Information: {retrieved_text}\nPlease classify this ticket into one of the following categories: Login Issues, App Functionality, Billing, Account Management, Performance Issues."
    
    response = ollama.chat(model="llama3.1:latest", messages=[{"role": "user", "content": prompt}])
    response_content = response['message']['content'].strip()
    return response_content

User Interface

In [4]:
# Define Gradio interface with customized styling
def gradio_interface(Ticket):
    classification = classify_ticket(Ticket)
    return classification

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(
        lines=4,
        placeholder="Enter the support ticket here...",
        elem_id="input-textbox",
        elem_classes=["custom-textbox"]
    ),
    outputs="text",
    title="Support Ticket Classification",
    description="Enter a support ticket to classify it into one of the predefined categories. The model will analyze the text and provide the most relevant category based on the knowledge base.",
    theme="default",
    css="""
        .gradio-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 100vh;
            background: url('https://www.getzephyr.com/zephyr/media/files/Technical-Support.png') no-repeat center center fixed;
            background-size: cover;
            padding: 20px;
        }
        #input-textbox {
            width: 100%;
            max-width: 600px;
            margin: 10px 0;
            background-color: transparent;
            border: 2px solid #ccc;
            border-radius: 8px;
            padding: 10px;
            font-size: 14px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .custom-textbox input {
            width: 100%;
            padding: 10px;
            border: #333;
            background: #333;
            font-size: 14px;
        }
        .output {
            width: 90%;
            height: 300px;
            max-width: 600px;
            margin: 10px 0;
            background-color: #fff;
            border: 2px solid #ccc;
            border-radius: 8px;
            padding: 10px;
            font-size: 14px;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .gradio-title {
            color: #333;
            font-size: 24px;
            margin-bottom: 10px;
        }
        .gradio-description {
            color: #666;
            font-size: 16px;
            margin-bottom: 20px;
        }
        .input-container {
            width: 90%;
            max-width: 600px;
            text-align: center;
            margin-bottom: 20px;
        }
        .input-label {
            display: block;
            font-size: 18px;
            font-weight: bold;
            color: #333;
            margin-bottom: 8px;
        }
        .gradio-button {
            background-color: #4CAF50;
            border: none;
            color: white;
            padding: 10px 16px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 14px;
            margin: 10px 0;
            cursor: pointer;
            border-radius: 8px;
        }
        .gradio-button:hover {
            background-color: #45a049;
        }
    """
)

# Launch the interface
iface.launch()

IMPORTANT: You are using gradio version 3.36.1, however version 4.29.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7917

To create a public link, set `share=True` in `launch()`.
